In [77]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import os

try:
    driver = webdriver.Chrome()
except:
    print("AF: No Chrome webdriver installed")
    driver = webdriver.Chrome(ChromeDriverManager().install())

try:
    url = "https://www.forexfactory.com/calendar?month=feb.2024"
    driver.get(url)
    while True:
        before_scroll = driver.execute_script("return window.pageYOffset;")
        driver.execute_script("window.scrollTo(0, window.pageYOffset + 500);")
        time.sleep(2)
        after_scroll = driver.execute_script("return window.pageYOffset;")
        if before_scroll == after_scroll:
            break

    # Get the table
    table = driver.find_element(By.CLASS_NAME, "calendar__table")
    data = []
    last_date = None
    last_time = None
    currency_codes = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']

    for row in table.find_elements(By.TAG_NAME, "tr"):
        row_data = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            if td.text:
                text = td.text.replace('\n', ', ')
                row_data.append(text)
                if ',' in text:  
                    last_date = text
                elif re.match(r'\d+:\d+[ap]m', text):
                    last_time = text
            else:
                if td.get_attribute("class") == "calendar__cell calendar__impact":
                    span_element = td.find_element(By.TAG_NAME, 'span')
                    row_data.append(span_element.get_attribute("title"))
        row_data = [x for x in row_data if x is not None] 
        if row_data and len(row_data) > 1:  
            if row_data[0] == 'Tentative':  
                row_data.insert(0, last_time)  
                row_data.insert(0, last_date)  
            elif row_data[0].startswith('Day'):  
                row_data.insert(0, last_date)  
            elif row_data[0] in currency_codes:  
                row_data.insert(0, last_time)  
                row_data.insert(0, last_date)  
            elif row_data[0] == 'All Day' or re.match(r'\d+:\d+[ap]m', row_data[0]):  
                row_data.insert(0, last_date)  
            data.append(row_data)
            print(row_data)

    second_list = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
    result_list = [[None] + row if len(row) > 1 and row[1] in second_list else row for row in data]
    result_list = [[None] + [None] + row if len(row) > 0 and row[0] in second_list else row for row in result_list]

    column_names = ['Date', 'Time', 'Currency', 'Impact', 'Event', 'Actual', 'Forecast', 'Previous']
    df = pd.DataFrame(result_list)
    df.columns = column_names

    month_year = url.split('=')[-1]
    year = month_year.split('.')[-1]

    directory = f'data/{year}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    df.to_csv(f'{directory}/{month_year}.csv', index=False)
    print("done")

except Exception as e:
    print(e)

finally:
    driver.quit()

['Thu, Feb 1', '2:00am', 'USD', 'High Impact Expected', 'Federal Funds Rate', '5.50%', '5.50%']
['Thu, Feb 1', '2:00am', 'USD', 'High Impact Expected', 'FOMC Statement']
['Thu, Feb 1', '2:30am', 'USD', 'High Impact Expected', 'FOMC Press Conference']
['Thu, Feb 1', '7:30am', 'AUD', 'Low Impact Expected', 'Building Approvals m/m', '0.1%', '1.6%']
['Thu, Feb 1', '7:30am', 'AUD', 'Low Impact Expected', 'Import Prices q/q', '0.6%', '0.8%']
['Thu, Feb 1', '7:30am', 'AUD', 'Low Impact Expected', 'NAB Quarterly Business Confidence', '-1']
['Thu, Feb 1', '7:30am', 'JPY', 'Low Impact Expected', 'Final Manufacturing PMI', '48.0', '48.0']
['Thu, Feb 1', '8:45am', 'CNY', 'Medium Impact Expected', 'Caixin Manufacturing PMI', '50.9', '50.8']
['Thu, Feb 1', '10:35am', 'JPY', 'Low Impact Expected', '10-y Bond Auction', '0.60|2.9']
['Thu, Feb 1', '12:30pm', 'AUD', 'Low Impact Expected', 'Commodity Prices y/y', '-10.7%']
['Thu, Feb 1', '3:15pm', 'EUR', 'Low Impact Expected', 'Spanish Manufacturing PMI',